In [17]:
import pickle
import pandas as pd
import cv2
import numpy as np
model = pickle.load(open('shit.pkl', 'rb'))

In [18]:
def get_rect_list(contours):
    rect_list = []
    temp = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        temp.append([x, 1 / h, y, w, h])

    temp.sort()
    last = -1
    for rect in temp:
        if rect[0] + rect[3] >= last and (rect[-1] > 30 or rect[-2] > 30):
            last = rect[0] + rect[3]
            rect_list.append([rect[0], rect[2], rect[3], rect[4]])
    return rect_list

def flat(img):
    temp = []
    for i in range(28):
        for j in range(28):
            temp.append(img[j][i])
    
    return temp

def normalize(img):

    diff = len(img[0]) - 28
    if diff == 0:
        # print(img)
        return img
    
    if diff > 0:

        right = (diff + 1) // 2
        left = (diff) // 2
        
        np.delete(img, np.s_[len(img[0]) - right : len(img[0])], axis = 1)
        np.delete(img, np.s_[0 : left], axis=1)
        return img

    diff *= -1

    left_zeros = np.zeros((28 , diff // 2), dtype=img.dtype)
    right_zeros = np.zeros((28, (diff + 1) // 2), dtype=img.dtype)

    img = np.concatenate((left_zeros, img, right_zeros), axis=1)
        
    # print (img)
    return img 

table = {
-1:63,
0: 48,
1: 49,
2: 50,
3: 51,
4: 52,
5: 53,
6: 54,
7: 55,
8: 56,
9: 57,
10: 65,
11: 66,
12: 67,
13: 68,
14: 69,
15: 70,
16: 71,
17: 72,
18: 73,
19: 74,
20: 75,
21: 76,
22: 77,
23: 78,
24: 79,
25: 80,
26: 81,
27: 82,
28: 83,
29: 84,
30: 85,
31: 86,
32: 87,
33: 88,
34: 89,
35: 90,
36: 97,
37: 98,
38: 100,
39: 101,
40: 102,
41: 103,
42: 104,
43: 110,
44: 113,
45: 114,
46: 116
}

In [27]:
all = []
def exec (imge):

  global all
  all = []

  img_read = cv2.imread(imge, cv2.IMREAD_GRAYSCALE)
  img_gas = cv2.GaussianBlur (img_read, (3, 3), 3)
  img_thresh2 = cv2.adaptiveThreshold(img_gas, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 181, 10)
  img_thresh2 = ~img_thresh2


  # cv2.imshow("trial",img_thresh2)
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()


  contours, hierarchy = cv2.findContours(img_thresh2,
    cv2.RETR_TREE,
    cv2.CHAIN_APPROX_SIMPLE)

  rect_list = get_rect_list (contours)

  # for rect in rect_list:
  #   x = rect[0]
  #   y = rect[1]
  #   w = rect[2]
  #   h = rect[3]
  #   cv2.rectangle(img_read ,(x,y), (x+w,y+h), (0,0,0), 5)
  #   cv2.imshow("img_read", img_read)
  #   cv2.waitKey(0)
  #   cv2.destroyAllWindows()

  final_pic=''
  for rect in rect_list:
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    roi = img_thresh2[y : y + h, x : x + w]
    
    desired_height = 28
    aspect_ratio = desired_height / roi.shape[0]
    desired_width = int(roi.shape[1] * aspect_ratio)
    dim = ( desired_width, desired_height) 

    
    resized_cropped_region = cv2.resize(roi, dsize=dim, interpolation=cv2.INTER_AREA)
    normal_pic = normalize (resized_cropped_region)
    temp = flat(normal_pic)
    final_pic = np.array(temp)
    all.append(final_pic)
  all = np.array(all)
  all = all/255
  all = all.reshape(-1, 28, 28, 1)
  

In [29]:

exec('char.png')
y_pred2 = model.predict(all)

def maxin (lis):

    maxim = -1
    ind = -1
    for i in range(len(lis)):
        if lis[i] > maxim and lis[i] >= 0.5:
            maxim = lis[i]
            ind = i
    return ind

for i in y_pred2:
    print(chr(table[maxin(i)]), end = " ")

1/1 [==============================] - 0s 29ms/step
A D F ? K M 0 R U W Z 